## TYTANチュートリアル（犯人は誰だ？）

2023年5月20日

ビネクラ安田

出典：[量子アニーリング（QUBO）で論理クイズ「犯人は誰だ？」を解く](https://vigne-cla.com/21-24/)

### 問題
論理パズルでよく見るやつ。

**【問題】<br>A, B, C, Dの中に犯人が１人います。犯人は嘘をつき、他の人は本当のことを証言します。犯人は誰でしょうか？**

*   A「僕とCは犯人ではない」
*   B「犯人はA, C, Dの誰か」
*   C「Eは犯人ではない」
*   D「A, C, Eは犯人ではない」
*   E「犯人はA, Bのどちらか」


### QUBOモデルでは何が設定できるか？（おさらい）

**「n個の量子ビットからm個を1にする」**

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

**「2個の量子ビットが同時に<font color="red">1</font>になったら報酬を与える」**

例）2個の量子ビットが同時に1になったら報酬を与える
```
H = -(q0 * q1)
```

**「2個の量子ビットが同時に<font color="red">0</font>になったら報酬を与える」**

例）2個の量子ビットが同時に0になったら報酬を与える
```
H = -(1 - q0) * (1 - q1)
```

今回はこれだけを使うが、その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

各人に5個の量子ビット（量子ビット名：A～E）を割り当て、１になったら犯人とする。

まず、5人の中に犯人が1人いるので「5個の量子ビットから1つだけ1になる」を設定する。この条件は必ず満たす必要があるので**強い条件**（重み1）である。
```
H = (A + B + C + D + E - 1)**2
```

次に、各人の証言を「もし～なら～が成り立つ」というif文を使って言い換える。「もし自分が正直者なら、自分の証言が成り立つ」という意味。

*   「もしAが0なら、A, Cは0」
*   「もしBが0なら、A, C, Dのどれかが1」
*   「もしCが0なら、Eは0」
*   「もしDが0なら、A, C, Eは0」
*   「もしEが0なら、A, Bのどれかが1」

さらにこれを「◯◯かつ✕✕なら報酬を与える」に言い換える。分配法則のようなイメージで。

*   A=0かつA=0なら報酬、A=0かつC=0なら報酬
*   B=0かつA=1なら報酬、B=0かつC=1なら報酬、B=0かつD=1なら報酬
*   C=0かつE=0なら報酬
*   D=0かつA=0なら報酬、D=0かつC=0なら報酬、D=0かつE=0なら報酬
*   E=0かつA=1なら報酬、E=0かつB=1なら報酬

例えば、「B=0かつC=1なら報酬」の部分は次のように設定できる。

```
H = -0.1 * ((1 - B) * C)
```

なお、これら証言の重みを1にしてしまうと報酬欲しさに犯人を2人にしてしまう可能性がある。したがってここは**弱い条件**（重み0.1）とする。

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [3]:
from sympy import Symbol
from tytan import qubo, sampler

#量子ビットを用意する
A = Symbol('qA')
B = Symbol('qB')
C = Symbol('qC')
D = Symbol('qD')
E = Symbol('qE')

#5人の中に犯人が1人いる（強い条件）
H = 0
H += (A + B + C + D + E - 1)**2

#各人の証言（成り立てば弱い報酬）
H += -0.1 * ((1 - A) * (1 - A))
H += -0.1 * ((1 - A) * (1 - C))

H += -0.1 * ((1 - B) * A)
H += -0.1 * ((1 - B) * C)
H += -0.1 * ((1 - B) * D)

H += -0.1 * ((1 - C) * (1 - E))

H += -0.1 * ((1 - D) * (1 - A))
H += -0.1 * ((1 - D) * (1 - C))
H += -0.1 * ((1 - D) * (1 - E))

H += -0.1 * ((1 - E) * A)
H += -0.1 * ((1 - E) * B)


#コンパイル
QUBO, offset = qubo.Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(QUBO, shots=500)

#結果
for r in result:
    print(r)

[{'qA': 0.0, 'qB': 1.0, 'qC': 0.0, 'qD': 0.0, 'qE': 0.0}, -1.1, 252]
[{'qA': 1.0, 'qB': 0.0, 'qC': 0.0, 'qD': 0.0, 'qE': 0.0}, -0.8999999999999998, 248]


犯人はBだとわかった。解のエネルギーは叶った報酬の数によるが、これは問題からは予想が難しいと思われる。

このままだとこのチュートリアルが文章だけになってしまうので、紙とペンで解く方法も図解する。

まず、「～が犯人だ」を「～は犯人ではない」に言い換える。次のように表を用意して、犯人ではないと証言された部分にNoを書き、Noの数を縦に集計する。Noが0個の人が犯人である。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-24_2.png" width = 45%>
</div>

ただ、通常はNoが0個の人はいない。そこで対角成分を無効にして集計する。これは犯人による「自分は犯人ではない」という証言を無効にするため。これで答えが得られるはず。